# Lab 5a: Final part on Optimisation

The purpose of this (short) first part of the lab session is to practice on duality (we did discuss in last week's lecture) and on sensitivity reports provided by the solvers we have seen so far (e.g., Excel, Gurobi, etc.)

## Duality

**Task 1:** Consider the following linear program
\begin{eqnarray*}
\min & \hspace{1cm}& 2x_1+3x_2 \\
\text{subject to} & \hspace{1cm}&x_1+x_2 \geq 5\\
& \hspace{1cm}&x_1 \geq 1\\
& \hspace{1cm}&x_2 \geq 2\\
\end{eqnarray*}

Prove that $x^*=(3,2)$ is the optimal solution by showing that the objective value of any feasible solution is at least $12$.

**Solution to Task 1:** Click on *Details* below to reveal it.

<details>

First, let's observe that by setting $x_1=3$ and $x_2=2$, the objective function $2x_1+3x_2$ takes the value $12$, while all constraints are respected. Then, to solve the task, we need to suitably combine the three constraints so that we prove the desired property, i.e., that the objective function must be at least $12$.
    
Note that, since $x_1+x_2 \geq 5$ from the first constraint, it holds that $2x_1+2x_2 = 2(x_1+x_2) \geq 10$. Furthermore, by the last constraint we have $x_2\geq 2$. By combining $2x_1+2x_2\geq 10$ and $x_2\geq 2$, we get that the objective function $2x_1+3x_2\geq 12$ as desired.
</details>

**Task 2:** Write the dual of the linear program in Task 1.

**Solution to Task 2:** Click on *Details* below to reveal it.

<details>
Recall the *high-level idea*, we need to find coefficients $y_1$, $y_2$, and $y_3$ and multiply the first constraint by $y_1$, the second constraint with $y_2$ and the third constraint with $y_3$. 
    
What is our goal? Recall we wish to prove a *lower bound* on the objective function $2x_1+3x_2$. Therefore, we must guarantee that the *weighted coefficients* of $x_1$ and $x_2$, as arising from the *dual*, must be at most $2$ and $3$ respectively. That is, it must hold that $y_1+y_2\leq 2$ (this is about $x_1$) and, similarly, $y_1+y_3 \leq 3$ (this is about $x_2$). Furthermore, we wish to prove an as high lower bound as possible, that is, we wish to *maximise* $5y_1+y_2+2y_3$. Furthermore, it must be that the $y_i$'s are non-negative (otherwise, it does not hold that $(x_1+x_2)\cdot y_1 \geq 5\cdot y_1$). 
    
To conclude, we get the following dual linear program:
    
\begin{eqnarray*}
\max & \hspace{1cm}& 5y_1+y_2+2y_3 \\
\text{subject to} & \hspace{1cm}&y_1+y_2 \leq 2\\
& \hspace{1cm}&y_1+y_3 \leq 3\\
& \hspace{1cm}& y_1, y_2 , y_3 \geq 0\\
\end{eqnarray*}
    
Observe that by setting $y_1=2$, $y_2=0$, and $y_3=1$, all constraints are respected and the objective function becomes $5y_1+y_2+2y_3 = 12$, as asked in Task 1. The fact that $y^* = (2,0,1)$ while in Task 1 we multiplied the first constraint by $2$ and the third by $1$ (while we ignored the second constraint) is *not a coincidence*.
</details>

## Sensitivity reports

 We will see how to generate sensitivity reports using again Task 5 from Lab 3. Click *Details* below to recall Task 5.

<details>
A company will face the following cash requirements in the next eight quarters (positive entries
represent cash needs while negative entries represent cash surpluses). 

| Q1 | Q2 | Q3 | Q4 | Q5 | Q6 | Q7 | Q8 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

The company has three borrowing possibilities.
<ul>
<li> a 2-year loan available at the beginning of Q1, with a 1% interest per quarter.
<li> The other two borrowing opportunities are available at the beginning of every quarter: a 6-
month loan with a 1.8% interest per quarter, and a quarterly loan with a 2.5% interest for the
quarter.
</ul>
    
Any surplus can be invested at a 0.5% interest per quarter. Formulate a linear program that maximises
the wealth of the company at the beginning of Q9. 
</details>

**Task 3:** Generate the sensitivity report for Task 5 with your favourite LP solver.

(i) Suppose the cash requirement in Q2 is 300 (instead of 500). How would this affect the wealth in Q9?

(ii) Suppose the cash requirement in Q2 is 100 (instead of 500). Can the sensitivity report be used to determine the wealth in Q9?


**Note:** In Excel Solver, you can generate the Sensitivity Report after you click on “Solve” as follows

![title](excel_sensitivity.png)

You can find an Excel file with the model already implemented [here](./sensitivity.xlsx)

**Solution to Task 3, using Excel:** Click *Details*

<details>
For (i), we see (by scrolling down) that the shadow price of cell $b$41 corresponding the cash requirement in Q2 is -1.066…. The allowable decrease is 307.152…. which means that we can use this shadow price when reducing the value of 500 to 300, since this decrease is 200 and not greater than the allowable decrease. So, we multiply the shadow price by the change in value (i.e., with -200) and get the change in the solution; the wealth will increase by that amount.

For (ii), the change in value (-400) is greater than the allowable decrease (which is 307.152…), so we cannot use the sensitivity report; we must change the LP and solve again.
</details>

**Solution to Task 3, using gurobipy:**
Gurobipy allows to compute the shadow prices by running **shadow_price = m.getAttr(GRB.Attr.Pi)** where m corresponds to the model. To get the lower and upper bounds on the allowable change, the relevant attributes are **SARHSLow** and **SARHSUp** respectively. It allows to compute the reduced cost by **shadow_price = m.getAttr(GRB.Attr.RC)**.

We saw the following code in Lab 4.

In [7]:
!pip install gurobipy --user
import numpy as np
from gurobipy import *
from gurobipy import GRB

# Declare and initialize model
m = Model()

# Add variables
x = m.addMVar(shape=22, vtype=GRB.CONTINUOUS, name="x")

# Create matrix A
A = np.zeros((8,22))

# I create a row for each constraint
A[0] = np.concatenate([[1,1,1,-1],np.zeros(18)])
A[1] = np.concatenate([[-0.01, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(15)])
A[2] = np.concatenate([[-0.01, -1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(12)])
A[3] = np.concatenate([[-0.01, 0, 0, 0, -1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(9)])
A[4] = np.concatenate([[-0.01], np.zeros(6), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(6)])
A[5] = np.concatenate([[-0.01], np.zeros(9), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(3)])
A[6] = np.concatenate([[-0.01], np.zeros(12), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, -1, 0]])
A[7] = np.concatenate([[-1.01], np.zeros(15), [-1.018, 0, 0, -1.025, 1.005, -1]])

# The right-hand-side constants
rhs = np.array([100, 100, 100, -600, -500, 200, 600, -900])

# Adding the constraints
m.addConstr(A @ x == rhs, name="c")

# Setting the objective function. Here, I can be flexible and define a maximization problem, unlike in linprog
obj = np.concatenate([[0]*21,[1]])
m.setObjective(obj @ x, GRB.MAXIMIZE)

m.optimize()
print(x.X)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7700 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 22 columns and 55 nonzeros
Model fingerprint: 0x58bf644f
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 9e+02]
Presolve time: 0.01s
Presolved: 8 rows, 22 columns, 55 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+30   1.951220e+30   1.000000e+00      0s
      10    8.9717070e+02   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.971706980e+02
[100.           0.           0.           0.         101.
   0.           0.           0.         102.818        0.
   0.           0.         390.79355      0.           0.
 891.747

Your task is to add a few lines in the code above, so that you can answer the two questions in Task 3.

Click *Details* to see the code that could be added above.
<details>

### We now get and print the shadow prices. 

shadow_price = m.getAttr(GRB.Attr.Pi)

print('shadow price: ', shadow_price)
    
low_allowable = m.getAttr(GRB.Attr.SARHSLow)

print('lower bounds on allowable change: ', low_allowable)

high_allowable = m.getAttr(GRB.Attr.SARHSUp)

print('upper bounds on allowable change: ', high_allowable)    

### Similarly for the reduced costs.

reduced_cost = m.getAttr(GRB.Attr.RC)

print('reduced cost: ', reduced_cost)
<\details>

In [10]:
shadow_price = m.getAttr(GRB.Attr.Pi)

print('shadow price: ', shadow_price)

low_allowable = m.getAttr(GRB.Attr.SARHSLow)

print('lower bounds on allowable change: ', low_allowable)

high_allowable = m.getAttr(GRB.Attr.SARHSUp)

print('upper bounds on allowable change: ', high_allowable)



shadow price:  [-1.071532398751384, -1.0573349863727808, -1.0456542631406245, -1.0201505006249996, -1.0150751249999996, -1.0100249999999997, -1.005, -1.0]
lower bounds on allowable change:  [0.0, -1.0, -2.817999999999998, -inf, -inf, -inf, -inf, -inf]
upper bounds on allowable change:  [937.2781813167383, 192.84729940252748, 193.88447167390206, -503.7684165342504, -403.2872586169217, 297.1963050899937, 697.6822866154437, -2.829301951479124]


In [9]:
reduced_cost = m.getAttr(GRB.Attr.RC)

print('reduced cost: ', reduced_cost)

reduced cost:  [0.0, -0.011975670880481681, -0.012235962280716217, -0.008910737446739425, 0.0, -0.014460633346359275, -0.006452451916453272, -0.006054923120625055, 0.0, -0.02040301001250011, -0.02632630162500016, -0.020301502499999957, 0.0, -0.026195325000000214, -0.020200500000000066, 0.0, -0.026065000000000338, -0.020100000000000007, 0.0, -0.020000000000000018, 0.0, 0.0]
